In [4]:
import pandas as pd
import numpy as np
import random
#from catboost import CatBoostRanker, Pool, MetricVisualizer
from copy import deepcopy

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
HotelSearch_orig=pd.read_csv("/Users/sureshvadakath/Documents/Datasets/SearchRanking/HotelSearch_train.csv")

In [7]:
HotelSearch_orig.dtypes
def get_target(row):
    """
    0=not clicked at all, 1=clicked but not booked, 2=booked
    """
#     if row.booking_bool>0:
#         return 2
#     if row.click_bool>0 :
#         return 1
#     return 0

    if row.booking_bool>0:
        return 1
    return 0

#print(HotelSearch_orig['booking_bool'].unique())
HotelSearch_orig["date_time"] = pd.to_datetime(HotelSearch_orig["date_time"],infer_datetime_format=True)
HotelSearch_orig["target"] = HotelSearch_orig.apply(get_target,axis=1)

HotelSearch_orig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9917530 entries, 0 to 9917529
Data columns (total 55 columns):
 #   Column                       Dtype         
---  ------                       -----         
 0   srch_id                      int64         
 1   date_time                    datetime64[ns]
 2   site_id                      int64         
 3   visitor_location_country_id  int64         
 4   visitor_hist_starrating      float64       
 5   visitor_hist_adr_usd         float64       
 6   prop_country_id              int64         
 7   prop_id                      int64         
 8   prop_starrating              int64         
 9   prop_review_score            float64       
 10  prop_brand_bool              int64         
 11  prop_location_score1         float64       
 12  prop_location_score2         float64       
 13  prop_log_historical_price    float64       
 14  position                     int64         
 15  price_usd                    float64       
 16  

In [8]:
HotelSearch_all=HotelSearch_orig.dropna(axis=1,how="all")
HotelSearch_all.shape


(9917530, 55)

In [9]:
All_Searches=HotelSearch_all["srch_id"].unique().tolist()
training_sample_list = random.sample(All_Searches, int(len(All_Searches)*.01))
print(len(All_Searches))
print(len(training_sample_list))
test_sample_list_temp=set(All_Searches) - set(training_sample_list)
test_sample_list = random.sample(test_sample_list_temp, int(len(test_sample_list_temp)*.015))
print(len(test_sample_list_temp))
print(len(test_sample_list))
#training_searches_array = np.array(training_sample_list)
#test_searches_array = np.array(test_sample_list)


399344
3993
395351
5930


/var/folders/kc/v7s5qpm17l14gx5297w1_6pw0000gn/T/ipykernel_87181/1782490068.py:6: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  test_sample_list = random.sample(test_sample_list_temp, int(len(test_sample_list_temp)*.015))


In [23]:
#HotelSearch_train = HotelSearch_all.groupby('srch_id').filter(lambda x: x["srch_id"].values[0] in sampled_searches)
HotelSearch_train_temp = HotelSearch_all[HotelSearch_all['srch_id'].isin(training_sample_list)]
# Not in Test:  position, click_bool, gross_bookings_usd, nor booking_bool
HotelSearch_test_temp = HotelSearch_all[HotelSearch_all['srch_id'].isin(test_sample_list)]
print(HotelSearch_train_temp.shape)
print(HotelSearch_test_temp.shape)



(98361, 55)
(147730, 55)


In [24]:
HotelSearch_train_temp=HotelSearch_train_temp.sort_values(by=['srch_id','position'], ascending=True)
HotelSearch_train_temp = HotelSearch_train_temp.reset_index(drop=True)
HotelSearch_train_temp.shape

(98361, 55)

In [25]:
HotelSearch_test_temp=HotelSearch_test_temp.sort_values(by=['srch_id','position'], ascending=True)
HotelSearch_test_temp = HotelSearch_test_temp.reset_index(drop=True)
HotelSearch_test_temp.shape

(147730, 55)

In [26]:
HotelSearch_train_temp.to_csv('./data/HotelSearch_train_sampled.csv')
HotelSearch_test_temp.to_csv('./data/HotelSearch_test_sampled.csv')
HotelSearch_train_temp.shape

(98361, 55)